# Load datasets to mLab

* https://www.kaggle.com/c/twitter-sentiment-analysis2/discussion
* https://www.kaggle.com/kazanova/sentiment140

In [1]:
import pymongo
import pandas as pd
import numpy as np
import random

In [2]:
def db_connection(collection_name): 
    # connect to mLab DB
    try:
        with open("../credentials/mlab_credentials.txt", 'r', encoding='utf-8') as f:
            [name,password,url,dbname]=f.read().splitlines()
            db_conn = pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
            print ("DB connected successfully!!!")
    except pymongo.errors.ConnectionFailure as e:
        print ("Could not connect to DB: %s" % e) 
        
    db = db_conn[dbname]
    collection = db[collection_name]
    
    return collection


def kaggle_id(id):
    return "kaggle_train_" + str(id)


def sentiment140_id(id):
    return "sentiment140_" + str(id)


def random_sentiment(sentiment):
    sentiments_list = [-2,-1,0,1,2]
    return random.choice(sentiments_list)


def random_location(dummy):
    location = random.choice(locations_list)
    #location = locations_list[0]
    lat = random_location_lat(location)
    lon = random_location_lon(location)
    return [lat,lon]

def random_location_lat(location):
    #location = random.choice(locations_list)
    #location = locations_list[0]
    lat = location["lat_min"] + random.random()*(location["lat_max"] - location["lat_min"])
    return lat

def random_location_lon(location):
    #location = random.choice(locations_list)
    #location = locations_list[0]
    lon = location["lon_min"] + random.random()*(location["lon_max"] - location["lon_min"])
    return lon

In [3]:
db_collection_locations = db_connection("twitter_happiness_locations")
locations_list = [location for location in db_collection_locations.find()]

DB connected successfully!!!


In [4]:
#locations_list

In [5]:
db_collection = db_connection("twitter_happiness_test")

DB connected successfully!!!


In [8]:
# uncomment to delete
#result = db_collection.delete_many({})
#print(result.deleted_count, " documents deleted")

199989  documents deleted


In [6]:
data_kaggle = pd.read_csv(
    "source/kaggle/train.csv",
    encoding='latin-1',
    header=0,
    names=["id_src","class","text"]
)


data_kaggle["id"] = data_kaggle["id_src"].apply(kaggle_id)
data_kaggle["class"] = data_kaggle["class"].apply(random_sentiment)
#data_kaggle["lat"] = data_kaggle["sentiment"].apply(random_location_lat)
#data_kaggle["lon"] = data_kaggle["sentiment"].apply(random_location_lon)
data_kaggle["loc"] = data_kaggle.apply(random_location, axis=1)
data_kaggle["lat"] = data_kaggle["loc"].apply(lambda loc: loc[0])
data_kaggle["lon"] = data_kaggle["loc"].apply(lambda loc: loc[1])


print(data_kaggle.shape)
data_kaggle.head()

(99989, 7)


,id_src,class,text,id,loc,lat,lon
0,1,0,is so sad for my APL frie...,kaggle_train_1,"[29.582283560127767, -95.2123669360277]",29.582284,-95.212367
1,2,-1,I missed the New Moon trail...,kaggle_train_2,"[-33.570346425730314, 151.21344609642875]",-33.570346,151.213446
2,3,-1,omg its already 7:30 :O,kaggle_train_3,"[45.47313525058968, -73.41354389931941]",45.473135,-73.413544
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...,kaggle_train_4,"[40.604535732046614, -73.72297983977568]",40.604536,-73.722980
4,5,1,i think mi bf is cheating on me!!! ...,kaggle_train_5,"[39.02454595409937, -76.98676381131986]",39.024546,-76.986764


In [7]:
db_collection.insert_many(data_kaggle.to_dict('records'))

In [8]:
print(db_collection.count())

99989


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


In [9]:
data_sentiment140 = pd.read_csv(
    "source/sentiment140/training.1600000.processed.noemoticon.csv",
    encoding='latin-1',
    header=None,
    names=["class","id_src","date","flag","user","text"]
)

data_sentiment140["id"] = data_sentiment140["id_src"].apply(kaggle_id)
data_sentiment140["class"] = data_sentiment140["class"].apply(random_sentiment)
#data_sentiment140["lat"] = data_sentiment140["sentiment"].apply(random_location_lat)
#data_sentiment140["lon"] = data_sentiment140["sentiment"].apply(random_location_lon)
data_sentiment140["loc"] = data_sentiment140.apply(random_location, axis=1)
data_sentiment140["lat"] = data_sentiment140["loc"].apply(lambda loc: loc[0])
data_sentiment140["lon"] = data_sentiment140["loc"].apply(lambda loc: loc[1])

data_sentiment140 = data_sentiment140.head(100000).copy()
print(data_sentiment140.shape)
data_sentiment140.head()

(100000, 10)


,class,id_src,date,flag,user,text,id,loc,lat,lon
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",kaggle_train_1467810369,"[41.36557063353792, 2.157612647313772]",41.365571,2.157613
1,-1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,kaggle_train_1467810672,"[45.50605609468136, -73.32406387925283]",45.506056,-73.324064
2,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kaggle_train_1467810917,"[34.08652157769123, -117.29293802932993]",34.086522,-117.292938
3,2,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,kaggle_train_1467811184,"[38.84224867762802, -77.18531654343933]",38.842249,-77.185317
4,-1,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",kaggle_train_1467811193,"[38.78221887701957, -77.1430162922284]",38.782219,-77.143016


In [10]:
db_collection.insert_many(data_sentiment140.to_dict('records'))

In [11]:
print(db_collection.count())

199989


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


In [12]:
for tweet in db_collection.find()[:10]:
    print(tweet)

{'_id': ObjectId('5c00008a25aafd0b15779a6c'), 'id_src': 8837, 'class': 2, 'text': " brokeback mountain was terrible.\n8838,0,Sentiment140,# @Catherine42 I wouldn't mind but I only had 1/2 a portion &amp; then left 1/2 the cream  just fruit for me then until my hols  x\n8839,1,Sentiment140,# @DeliciousLunch ... dark chocolate cookies? oh you tease! I'm writing to day n dipping into twitter for company \n8840,1,Sentiment140,# followfriday @mstuyvenberg @feb_unsw @hazelmail @beckescreet - all almost as cool as he-man and she-ra \n8841,1,Sentiment140,# followfriday @presentsqueen because she talks sense \n8842,1,Sentiment140,# New York is the most amazing city i've ever been to \n8843,0,Sentiment140,# number times I bottomed out just in our driveway = 4... a 6.5 hour trip to mass.. I'm scared \n8844,0,Sentiment140,# of NYC celebrity street vendors &gt; # of POA celebrities  http://streetvendor.org/media/pdfs/Side2.pdf\n8845,1,Sentiment140,###### yay ##### thanks @matclayton ##### \n8846,0,

In [13]:
print(db_collection.count())

199989


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


In [3]:
db_collection_tweets = db_connection("tweets")

DB connected successfully!!!


In [4]:
for tweet in db_collection_tweets.find()[:]:
    #print(tweet)
    print(tweet[""])

KeyError: ''

In [22]:
type(db_collection_tweets.find()[0]["created_at"])

str

In [9]:
location_query = {
    "coordinates": {
        "$ne": None
    }
}

In [13]:
for tweet in db_collection_tweets.find(location_query):
    print(tweet["coordinates"])

{'type': 'Point', 'coordinates': [-73.98044169, 40.77817731]}
{'type': 'Point', 'coordinates': [-87.62362195, 41.88270986]}
{'type': 'Point', 'coordinates': [-95.60276, 30.01138]}
{'type': 'Point', 'coordinates': [-95.61707723, 29.91684185]}
{'type': 'Point', 'coordinates': [-95.3698028, 29.7604267]}
{'type': 'Point', 'coordinates': [-71.09973744, 42.34392621]}
{'type': 'Point', 'coordinates': [-71.06138382, 42.36177402]}
{'type': 'Point', 'coordinates': [-71.03925, 42.37274]}
{'type': 'Point', 'coordinates': [-71.059773, 42.358431]}
{'type': 'Point', 'coordinates': [-71.0150608, 42.36576817]}
{'type': 'Point', 'coordinates': [-71.067605, 42.3016305]}
{'type': 'Point', 'coordinates': [-71.0588801, 42.3600825]}
{'type': 'Point', 'coordinates': [-71.0554013, 42.354474]}
{'type': 'Point', 'coordinates': [-71.0588801, 42.3600825]}
{'type': 'Point', 'coordinates': [-71.0588801, 42.3600825]}
{'type': 'Point', 'coordinates': [-71.0588801, 42.3600825]}
{'type': 'Point', 'coordinates': [-71.061